In [1]:
import folium

print(folium.__file__)
print(folium.__version__)

/usr/local/lib/python2.7/dist-packages/folium/__init__.pyc
0.2.1


In [2]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

LAND = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                    edgecolor='face',
                                    facecolor=cfeature.COLORS['land'])

def make_map(bbox, projection=ccrs.Mercator()):
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.add_feature(LAND, facecolor='0.25')
    ax.coastlines(resolution='50m')
    return fig, ax

In [4]:
import numpy.ma as ma
from scipy.io import loadmat
#from oceans.colormaps import cm

merc = loadmat('./mercator_temperature.mat', squeeze_me=True)

x, y = merc['x'], merc['y']
bbox = [x.min(), x.max(), y.min(), y.max()]

level = 10  # 10 meters temperature.
data = ma.masked_invalid(merc['temp'][level, ...])

fig, ax = make_map(bbox=bbox)
cs = ax.pcolormesh(x, y, data, cmap=plt.cm.ocean, transform=ccrs.PlateCarree())
fig.savefig('GS.png', transparent=True)

/usr/local/lib/python2.7/dist-packages/cartopy/io/__init__.py:264: DownloadWarning: Downloading: http://naciscdn.org/naturalearth/50m/physical/ne_50m_land.zip
  warnings.warn('Downloading: {}'.format(url), DownloadWarning)
/usr/local/lib/python2.7/dist-packages/cartopy/io/__init__.py:264: DownloadWarning: Downloading: http://naciscdn.org/naturalearth/50m/physical/ne_50m_coastline.zip
  warnings.warn('Downloading: {}'.format(url), DownloadWarning)


In [12]:
data.shape

(83, 103)

In [5]:
import numpy as np
from PIL import Image, ImageChops


def trim(img):
    border = Image.new(img.mode, img.size, img.getpixel((0, 0)))
    diff = ImageChops.difference(img, border)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        img = img.crop(bbox)
    return np.array(img)

img = Image.open("GS.png")
img = trim(img)

In [6]:
from folium import plugins


mapa = folium.Map(location=[y.mean(), x.mean()], zoom_start=5)

plugins.ImageOverlay(img,
                     [[y.min(), x.min()], [y.max(), x.max()]],
                     opacity=0.5).add_to(mapa)

mapa

Bypass the figure creation entirely:

In [7]:
import matplotlib.pyplot as plt

def colorize(array, cmap='rainbow'):
    normed_data = (array - array.min()) / (array.max() - array.min())
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)

colored_data = colorize(data, cmap='rainbow')

In [8]:
mapa = folium.Map(location=[y.mean(), x.mean()], zoom_start=5)

plugins.ImageOverlay(np.flipud(colored_data),
                     [[y.min(), x.min()], [y.max(), x.max()]],
                     opacity=0.5).add_to(mapa)

mapa

In [13]:
np.flipud(colored_data)


array([[[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.31568627,  0.95838122,  0.80171428,  1.        ],
        [ 0.29215686,  0.94717736,  0.81262237,  1.        ],
        [ 0.2372549 ,  0.9160336 ,  0.83698911,  1.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.21372549,  0.9005867 ,  0.84695821,  1.        ],
        [ 0.24509804,  0.92090552,  0.83360239,  1.        ],
        [ 0.18235294,  0.87808125,  0.85979985,  1.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.14313725,  0

In [14]:
[[y.min(), x.min()], [y.max(), x.max()]]

[[31.028392791748047, -78.0], [41.978797912597656, -61.0]]